<a href="https://colab.research.google.com/github/parrisem/Facial-Landmark-Detection/blob/main/Facial_Landmark_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import  History
from sklearn.cluster import KMeans
import skimage
from skimage import transform, util

# Data Loading

In [ ]:
!wget "http://users.sussex.ac.uk/~is321/training_images.npz" -O training_images.npz
# The test images (without points)
!wget "http://users.sussex.ac.uk/~is321/test_images.npz" -O test_images.npz
# The example images are here
!wget "http://users.sussex.ac.uk/~is321/examples.npz" -O examples.npz

# Load the data using np.load
data = np.load('training_images.npz', allow_pickle=True)
test = np.load('test_images.npz', allow_pickle=True)
examples = np.load('examples.npz', allow_pickle=True)

# Extract the images and points
images = data['images']
pts = data['points']
test_imgs = test['images']
example_imgs = examples['images']


# Visualising the images and points

In [ ]:
# Displays some of the images from the data
print(images.shape, pts.shape)
def visualise_pts(img, pts):
  import matplotlib.pyplot as plt
  plt.imshow(img)
  plt.plot(pts[:, 0], pts[:, 1], '+r')
  plt.show()

for i in range(3):
  idx = np.random.randint(0, images.shape[0])
  visualise_pts(images[idx, ...], pts[idx, ...])
 

# Calculating Prediction Error and exporting results

In [ ]:
def euclid_dist(pred_pts, gt_pts):
  """
  Calculate the euclidean distance between pairs of points
  :param pred_pts: The predicted points
  :param gt_pts: The ground truth points
  :return: An array of shape (no_points,) containing the distance of each predicted point from the ground truth
  """
  import numpy as np
  pred_pts = np.reshape(pred_pts, (-1, 2))
  gt_pts = np.reshape(gt_pts, (-1, 2))
  return np.sqrt(np.sum(np.square(pred_pts - gt_pts), axis=-1))

In [ ]:
def save_as_csv(points, location = '.'):
  """
  Save the points out as a .csv file
  :param points: numpy array of shape (no_image, no_points, 2) to be saved
  :param location: Directory to save results.csv in. Default to current working directory
  """
  np.savetxt(location + '/results.csv', np.reshape(points, (points.shape[0], -1)), delimiter=',')


# Pre-processing Images and Points

In [ ]:
# Making the training and testing images grayscale
copy_train_imgs = np.copy(images)
copy_train_pts = np.copy(pts)
copy_test_imgs = np.copy(test_imgs)

print("Training images:",copy_train_imgs.shape)
print("Training points:",copy_train_pts.shape)
print("Testing images:",copy_test_imgs.shape)

grey_train_imgs = []
grey_test_imgs = []

# training images
for i in range(0,copy_train_imgs.shape[0]):
  grey_train_imgs.append(cv2.cvtColor(copy_train_imgs[i], cv2.COLOR_BGR2GRAY))
grey_train_imgs = np.array(grey_train_imgs)
print("Grey training images:",grey_train_imgs.shape)

# testing images
for i in range(0,copy_test_imgs.shape[0]):
  grey_test_imgs.append(cv2.cvtColor(copy_test_imgs[i],cv2.COLOR_BGR2GRAY))
grey_test_imgs = np.array(grey_test_imgs)
print("Grey testing images:",grey_test_imgs.shape)


plt.imshow(grey_train_imgs[5])
plt.plot(copy_train_pts[5][:,0],copy_train_pts[5][:,1],'+r')
plt.show()


In [ ]:
# Rescaling the training and testing images and training points.
resc_gray_imgs = []
resc_pts = []
resc_gray_test_imgs = []

for i in range(0,grey_train_imgs.shape[0]):
  resc_gray_imgs.append(cv2.resize(grey_train_imgs[i],(100,100)))
for i in range(0,copy_train_pts.shape[0]):
  resc_pts.append(copy_train_pts[i]*[100/250,100/250])

for i in range(0,grey_test_imgs.shape[0]):
  resc_gray_test_imgs.append(cv2.resize(grey_test_imgs[i],(100,100)))

resc_gray_imgs = np.array(resc_gray_imgs)
resc_pts = np.array(resc_pts)
resc_gray_test_imgs = np.array(resc_gray_test_imgs)

print("Rescaled grey image:",resc_gray_imgs.shape)
print("Rescaled grey image points:",resc_pts.shape)
print("Rescaled grey test image:",resc_gray_test_imgs.shape)

plt. imshow(resc_gray_imgs[5])
plt.plot(resc_pts[5][:,0], resc_pts[5][:,1],'+r')
plt.show()

In [ ]:
# normalise training and testing image arrays.
norm_train_imgs = resc_gray_imgs/255
norm_train_pts = resc_pts/100-1
norm_test_imgs = resc_gray_test_imgs/255

plt.imshow(norm_train_imgs[5])
plt.plot((norm_train_pts[5][:,0]+1)*100,(norm_train_pts[5][:,1]+1)*100,'+r')
plt.show()

# CNN model

In [ ]:
history = History()
train_imgs_x = norm_train_imgs.reshape(norm_train_imgs.shape[0],100,100,1)
train_imgs_y = norm_train_pts.reshape(norm_train_pts.shape[0],-1)

In [ ]:
# first model
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', strides=1, input_shape=(100, 100, 1)))
model.add(MaxPooling2D(pool_size =(2,2)))

model.add(Conv2D(64,(3,3), padding='same', strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size =(2,2)))

model.add(Conv2D(64, (3,3), padding='same',strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size =(2,2)))

model.add(Conv2D(128,(3,3), padding='same',strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size =(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(84))
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
model.summary()


# Training the model

In [ ]:
history = model.fit(train_imgs_x, train_imgs_y, validation_split=0.3, batch_size=50, shuffle=True, epochs=150, verbose=1)

In [ ]:
# epochs 100
# loss plot
plt.figure(figsize=(17,7))
plt.subplot(1,2,1)
plt.suptitle('Original Data', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.plot(history.history['loss'], color='r', label='Training Loss')
plt.plot(history.history['val_loss'], color='b', label='Validation Loss')
plt.legend(loc='upper right')

# accuracy plot
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.plot(history.history['accuracy'], color='r', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], color='b', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
#epochs 150
# loss plot
plt.figure(figsize=(17,7))
plt.subplot(1,2,1)
plt.suptitle('Original Data', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.plot(history.history['loss'], color='r', label='Training Loss')
plt.plot(history.history['val_loss'], color='b', label='Validation Loss')
plt.legend(loc='upper right')

# accuracy plot
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.plot(history.history['accuracy'], color='r', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], color='b', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

# Predicting and saving the test images points

In [ ]:
pts_test = model.predict(norm_test_imgs.reshape(norm_test_imgs.shape[0], 100,100,1))
test_pts = pts_test.reshape(pts_test.shape[0],42,-1)
save_as_csv(test_pts)

In [ ]:
#vizualise some of the test images with thier predicted points, epochs 100
plt.imshow(norm_test_imgs[444])
plt.plot((test_pts[444][:,0]+1)*100,(test_pts[444][:,1]+1)*100, '+r')
plt.show()

In [ ]:
#vizualise some of the test images with thier predicted points, epochs 150
plt.imshow(norm_test_imgs[300])
plt.plot((test_pts[300][:,0]+1)*100,(test_pts[300][:,1]+1)*100, '+r')
plt.show()

# Augmenting the training data

In [ ]:
# Rotating the images and points
def rotate_img_pts(img,pts):
   """
  Rotate the images and their points
  :param img: The image
  :param pts: The points
  :return: Tuple of the rotated images and the rotated points
  """
  new_img = np.copy(img)
  rot_pts = np.copy(pts)
  rot_img = skimage.transform.rotate(new_img, angle=90)
  for i in range(0,42):
    rot_pts = np.rot90(pts,2)

  return rot_img,rot_pts


In [ ]:
rot_train_imgs = []
rot_train_pts = []

for i in range(0,norm_train_imgs.shape[0]):
  rotated_imgs,rotated_pts = rotate_img_pts(norm_train_imgs[i],norm_train_pts[i])
  rot_train_imgs.append(rotated_imgs)
  rot_train_pts.append(rotated_pts)

rot_train_imgs = np.array(rot_train_imgs)
rot_train_pts = np.array(rot_train_pts)


In [ ]:
plt.imshow(rot_train_imgs[3])
plt.plot((rot_train_pts[3][:,0]+1)*100,(rot_train_pts[3][:,1]+1)*100,'+r')
plt.show()

In [ ]:
new_training_imgs = np.concatenate((norm_train_imgs, rot_train_imgs))
new_training_pts = np.concatenate((norm_train_pts, rot_train_pts))
print(new_training_imgs.shape)
print(new_training_pts.shape)

In [ ]:
f_history = History()
f_train_imgs_x = new_training_imgs.reshape(new_training_imgs.shape[0],100,100,1)
f_train_imgs_y = new_training_pts.reshape(new_training_pts.shape[0],-1)

# Building final model

In [ ]:
# final model

f_model = Sequential()
f_model.add(Conv2D(32, (3,3), activation='relu', strides=1, input_shape=(100, 100, 1)))
f_model.add(MaxPooling2D(pool_size =(2,2)))

f_model.add(Conv2D(64,(3,3), strides=1, activation='relu'))
f_model.add(MaxPooling2D(pool_size =(2,2)))
f_model.add(Conv2D(64, (3,3), strides=1, activation='relu'))
f_model.add(MaxPooling2D(pool_size =(2,2)))

f_model.add(Conv2D(128,(3,3), strides=1, activation='relu'))
f_model.add(MaxPooling2D(pool_size =(2,2)))

f_model.add(Flatten())
f_model.add(Dense(512, activation='relu'))
f_model.add(Dropout(0.2))

f_model.add(Dense(84))
f_model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
f_model.summary()

# Training final model

In [ ]:
f_history = f_model.fit(f_train_imgs_x,f_train_imgs_y, validation_split=0.3, batch_size=50, shuffle=True, epochs=150, verbose=1)

In [ ]:
# Epochs 100
# loss plot
plt.figure(figsize=(17,7))
plt.subplot(1,2,1)
plt.suptitle('Rotated Data', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.plot(f_history.history['loss'], color='r', label='Training Loss')
plt.plot(f_history.history['val_loss'], color='b', label='Validation Loss')
plt.legend(loc='upper right')

# accuracy plot
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.plot(f_history.history['accuracy'], color='r', label='Training Accuracy')
plt.plot(f_history.history['val_accuracy'], color='b', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Epochs 150
# loss plot
plt.figure(figsize=(17,7))
plt.subplot(1,2,1)
plt.suptitle('Rotated Data', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.plot(f_history.history['loss'], color='r', label='Training Loss')
plt.plot(f_history.history['val_loss'], color='b', label='Validation Loss')
plt.legend(loc='upper right')

# accuracy plot
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.plot(f_history.history['accuracy'], color='r', label='Training Accuracy')
plt.plot(f_history.history['val_accuracy'], color='b', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

# Predicting and saving the test points for final model

In [ ]:
pts_test_2 = f_model.predict(norm_test_imgs.reshape(norm_test_imgs.shape[0], 100,100,1))
test_pts_2 = pts_test_2.reshape(pts_test_2.shape[0],42,-1)
save_as_csv(test_pts_2)

In [ ]:
#vizualise some of the test images with thie predicted points, epochs 100
plt.imshow(norm_test_imgs[300])
plt.plot((test_pts_2[300][:,0]+1)*100,(test_pts_2[300][:,1]+1)*100,'+r')
plt.show()

In [ ]:
#vizualise some of the test images with thie predicted points, epochs 150
plt.imshow(norm_test_imgs[300])
plt.plot((test_pts_2[300][:,0]+1)*100,(test_pts_2[300][:,1]+1)*100,'+r')
plt.show()

# Calculating Prediction Error and exporting results

In [ ]:
# Example 1
ed = f_model.predict(f_train_imgs_x)
ed = ed.reshape(ed.shape[0],42,-1)

In [ ]:
plt.imshow(norm_train_imgs[1])
plt.plot((norm_train_pts[1][:,0]+1)*100,(norm_train_pts[1][:,1]+1)*100,'+r')
plt.show()

euclid_dist((ed[1]+1)*100,(norm_train_pts[1]+1)*100)

In [ ]:
# Example 2
plt.imshow(norm_train_imgs[7])
plt.plot((norm_train_pts[7][:,0]+1)*100,(norm_train_pts[7][:,1]+1)*100,'+r')
plt.show()

euclid_dist((ed[7]+1)*100,(norm_train_pts[7]+1)*100)

In [ ]:
# Example 3
plt.imshow(norm_train_imgs[133])
plt.plot((norm_train_pts[133][:,0]+1)*100,(norm_train_pts[133][:,1]+1)*100,'+r')
plt.show()

euclid_dist((ed[133]+1)*100,(norm_train_pts[133]+1)*100)

# Face Segmentation System

In [ ]:
def img_segmentation(img,pts):
  """
  Fills the face in an image with colour using the points predicted by the model as its boundary
  :param img: The image
  :param pts: The predicted points
  """
  seg_img = np.zeros((100,100), dtype=np.uint8)
  bounds = pts

  fig, (ax1, ax2) = plt.subplots(1, 2)
  fig.suptitle('BEFORE & AFTER')
  cv2.fillPoly(seg_img, np.int32([bounds]), [255,255,255], lineType=4, shift=0)

  ax2.imshow(seg_img)
  ax1.imshow(img)


# Testing final model on example images

In [ ]:
# visualising the example images
example_imgs.shape

plt.imshow(example_imgs[1])
plt.show()

# Pre-processing example images

In [ ]:
# pre-processing the example images like I did with the training and testing images

gray_example_imgs = []
resc_example_imgs = []

# make grayscale
for i in range(0,example_imgs.shape[0]):
  gray_example_imgs.append(cv2.cvtColor(example_imgs[i],cv2.COLOR_BGR2GRAY))
gray_example_imgs = np.array(gray_example_imgs)
print("Grey example images:",gray_example_imgs.shape)

# resize
for i in range(0,gray_example_imgs.shape[0]):
  resc_example_imgs.append(cv2.resize(gray_example_imgs[i],(100,100)))
resc_example_imgs = np.array(resc_example_imgs)
print("Rescaled grey example images:", resc_example_imgs.shape)
plt.imshow(resc_example_imgs[1])

# normalize
norm_ex_imgs = resc_example_imgs/255
print("Normalized grey, rescaled example images:", norm_ex_imgs.shape)

plt.imshow(norm_ex_imgs[1])
plt.show()

# Predicting and saving the points for example images

In [ ]:
example_predict_pts = f_model.predict(norm_ex_imgs.reshape(norm_ex_imgs.shape[0],100,100,1))
ex_predict_pts = example_predict_pts.reshape(example_predict_pts.shape[0],42,2)
save_as_csv(ex_predict_pts)

In [ ]:
# visualising the points on the image
plt.imshow(norm_ex_imgs[1])
plt.plot((ex_predict_pts[1][:,0]+1)*100,(ex_predict_pts[1][:,1]+1)*100,'+r')
plt.show()

In [ ]:
# visualising the points on the image
plt.imshow(norm_ex_imgs[2])
plt.plot((ex_predict_pts[2][:,0]+1)*100,(ex_predict_pts[2][:,1]+1)*100,'+r')
plt.show()

In [ ]:
# visualising the points on the image
plt.imshow(norm_ex_imgs[3])
plt.plot((ex_predict_pts[3][:,0]+1)*100,(ex_predict_pts[3][:,1]+1)*100,'+r')
plt.show()

# Applying Face Segmentation on example images

In [ ]:
img_seg(example_imgs[1],(ex_predict_pts[1]+1)*100)

In [ ]:
img_seg(example_imgs[0],(ex_predict_pts[0]+1)*100)